Import Required Packages

In [19]:
#|export
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException
import psutil
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
import wd_classes
from wd_classes import FormEntry
import importlib
importlib.reload(wd_classes)
from wd_classes import FormEntry

from openpyxl import load_workbook

Get current open chrome browser

In [2]:
#|export
chrome_options = Options()
chrome_options.add_experimental_option("debuggerAddress", "127.0.0.1:9222")

#Change chrome driver path accordingly
#Before doing this launch chrome using cmd prompt command: chrome.exe --remote-debugging-port=9222 --user-data-dir="C:\Users\JoshG\Documents\Machine Learning\JobApp_Automation\ChromeProfile"
chrome_driver = "C:/Users/JoshG/AppData/Local/Microsoft/WindowsApps/chromedriver.exe"

driver = webdriver.Chrome(chrome_driver, chrome_options=chrome_options)

C:\Users\JoshG\AppData\Local\Temp\ipykernel_15696\1268542835.py:9: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(chrome_driver, chrome_options=chrome_options)


Error Handling

In [3]:
#errors

class stuck_on_login_page(Exception):
    print(f'page did not move past login page')
    pass

class stuck_on_create_account(Exception):
    print(f'Account not created')
    pass

class invalid_form_entry_type(Exception):
    print(f'answer type is not valid')

page did not move past login page
Account not created
answer type is not valid


Functions

In [4]:
#get login info saved in csv database
def wd_get_login_info(company_name,database="login_info.csv"):
    #open csv database
    df = pd.read_csv(database,dtype={'Password': str})
    
    #get email and password
    email = df.loc[df["Company"] == company_name, 'Email']
    email = email.values[0]
    password = df.loc[df["Company"] == company_name, 'Password']
    password = password.values[0]

    return email, password

In [5]:
email, password = wd_get_login_info('Test Company')
print(f'email: {email}, password {password}')


email: test@test.com, password 123456


In [6]:
#get login info saved in csv database
def write_login_info(company_name,email,password,database="login_info.csv"):
    #open csv database
    df = pd.read_csv(database)

    # add a new row to the DataFrame with the new person's name and company name
    new_row = pd.DataFrame({'Company': [f'{company_name}'], 'Email': [f'{email}'], 'Password': [f'{password}']})

    # concatenate the new DataFrame with the existing DataFrame
    df = pd.concat([df, new_row], ignore_index=False)

    # write the updated DataFrame to the existing CSV file
    df.to_csv('login_info.csv', index=False)

In [7]:
#check if this url has a workday account in database
def wd_check_account_exists(company_name, login_info="login_info.csv"):
    df = pd.read_csv(login_info,dtype={'Password': str})
    if company_name in df["Company"].values:
        return True
    else:
        return False

In [8]:
def check_page_type(page_type):
    #check to see if on page = "page_type"
    try:
        page_title_obj = driver.find_element_by_xpath(f'//h2[text()="{page_type}"]')
    except:
        return False
    #page_title_obj = driver.find_element_by_id("authViewTitle")
    page_title = page_title_obj.text
    print(page_title)
    if page_title == page_type:
        return True
    else:
        return False

In [9]:
#create new wd account
def wd_create_account(company_name, username, password):
    #get form items
    email_text_box = driver.find_element(By.XPATH, "//*[contains(text(), 'Email Address')]/following::input[@type='text'][1]")
    password_text_box = driver.find_element_by_css_selector("input[data-automation-id='password']")
    verify_password_text_box = driver.find_element_by_css_selector("input[data-automation-id='verifyPassword']")
    terms_checkbox = driver.find_element_by_css_selector("input[data-automation-id='createAccountCheckbox']")

    #fill in form
    email_text_box.clear()
    email_text_box.send_keys(email)

    #for some reason clear doesn't work with this field, so alternative is highlighting the text to clear
    password_text_box.send_keys(Keys.CONTROL,"a", Keys.DELETE)
    password_text_box.send_keys(password)
    password_text_box.send_keys(Keys.TAB)

    #See if password was accepted
    try:
        driver.find_element_by_css_selector("p[data-automation-id='inputError']")
        error_msg = driver.find_element_by_css_selector("p[data-automation-id='inputError']")
        if error_msg.is_displayed():
            print("Passsword has error")
        else:
            print("password valid, but inputerror label exists")

    except NoSuchElementException:
        print("Password is valid")

    verify_password_text_box.clear()
    verify_password_text_box.send_keys(password)

    #check terms and conditions checkbox
    if terms_checkbox.get_attribute("checked") == "true":
        print('was already checked')
    else:
        terms_checkbox.click()

    #Get create account button
    create_account_button = driver.find_element_by_xpath("//div[@aria-label='Create Account']")
    driver.implicitly_wait(1)
    driver.execute_script("arguments[0].click();", create_account_button)

    #Add to database if account created successfully
    if check_page_type('Create Account'):
        raise stuck_on_create_account("Did not move on to next page from create account")
    else:
        write_login_info(company_name, username, password)

In [10]:
def wd_sign_in(email, password):
    # Get email and password elements
    email_text_box = driver.find_element(By.XPATH, "//*[contains(text(), 'Email Address')]/following::input[@type='text'][1]")
    password_text_box = driver.find_element(By.XPATH, "//*[contains(text(), 'password')]/following::input[@type='password'][1]")
    
    
    #Fill in email and password
    email_text_box.send_keys(email)
    password_text_box.send_keys(password)

    #wait for a moment
    driver.implicitly_wait(2)
    
    #Get sign in button and click
    sign_in_button = driver.find_element_by_xpath("//div[@aria-label='Sign In']")
    driver.implicitly_wait(1)
    driver.execute_script("arguments[0].click();", sign_in_button)

    driver.implicitly_wait(1)

    
    #check to see if still on same page
    if check_page_type('login'):
        raise stuck_on_login_page("Login Info Didn't Work")
    else:
        print("no errors found on login page")
        


In [11]:
def get_current_company():
    #get url and pull the text after the first / after myworkdayjobs.com
    current_url = driver.current_url

    # Split the URL by "/"
    parts = current_url.split("/")

    # Get the text following ".com/" and before the next slash
    company_name = parts[3]

    return company_name

In [12]:
def wd_login_page(new_email,new_password):
    
    wait = WebDriverWait(driver, 20)
    wait.until(EC.presence_of_element_located((By.ID, "input-4")))
    
    current_company = get_current_company()

    #check if account exists for this site and login if has account or create account and then login
    if wd_check_account_exists(current_company) is True:
        email, password = wd_get_login_info(current_company)
        wd_sign_in(email, password)
    else:
        create_account_button = driver.find_element_by_xpath("//button[contains(text(), 'Create Account')]")
        create_account_button.click()
        
        wd_create_account(current_company, new_email, new_password)

In [13]:
def wd_startYourApplication_page():
    
    wait = WebDriverWait(driver, 20)
    wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "[data-automation-id='applyManually']")))

    manual_apply_btn = driver.find_element_by_css_selector("a[data-automation-id='applyManually']")
    manual_apply_btn.click()

def get_question_list():
    pass

def wd_myInformation_page():
    get_question_list()
    


In [14]:
def answer_question(element,question_label,answer,answer_type='textbox'):
    if answer_type == 'textbox':
        element.send_keys(answer)
    elif answer_type == 'button':
        print("funcitonality not implemnted yet")

In [138]:
#|export
url = 'https://jci.wd5.myworkdayjobs.com/JCI/login?redirect=%2FJCI%2Fjob%2FMissoula-Montana-United-States-of-America%2FHVAC-Truck-Base-Mechanic--Union-_WD30167702%2Fapply%3Fsource%3DCareers_Section'
email = "pyEngineer69@outlook.com"
password = "aTestPass@1234"

#get job posting url
driver.get(url)

#login or create account and log in
wd_login_page(email, password)

#Click on start manual application
try:
    wd_startYourApplication_page()
except:
    pass

no errors found on login page


In [39]:
def get_answer(df,question):
    #pull answer from form
    try:
        index = df.index[df['Questions'] == question].tolist()[0]
        answer = df.at[index, 'Answers']
    except:
        answer = ""
        print(f"failed to get answer for: {question}")

    if pd.isna(answer):
        answer = ""

    return answer

In [43]:
def load_page_by_dataautomationid(dataautomationid,t=20):
    wait = WebDriverWait(driver, 20)
    wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, f"[data-automation-id='{dataautomationid}']")))
    driver.implicitly_wait(0.5)

In [67]:
import wd_classes
from wd_classes import FormEntry
import importlib
importlib.reload(wd_classes)
from wd_classes import FormEntry

In [68]:
#|export
#
# #1st page of entry forms

load_page_by_dataautomationid('contactInformationPage')

#get question/answer database
df = pd.read_excel('Answers.xlsx',sheet_name="Answers",dtype=str)
#display(df.head(5))

# locate all input elements on the page
inputs_list = driver.find_elements_by_css_selector("[id^='input-']")
print(f'length of # entrys: {len(inputs_list)}')

entries = []
#move through each form entry and fill in
for element in inputs_list:
    
    #create element object
    entry = FormEntry(element)
    
    entry.get_id()
    entry.get_label()
    entry.get_tag()
    entry.get_answer_type()

    #skip if a span element
    if entry.tag == 'span':
        continue

    #pull answer from form
    answer = get_answer(df,entry.label)
    entry.add_answer(answer)

    print(f"form element prior to writing answer: {entry}")

    entry.write_answer(driver)
        
    #append entry to entries list
    entries.append(entry)
    driver.implicitly_wait(0.2)

continue_button = driver.find_element_by_xpath(f"//button[@data-automation-id='bottom-navigation-next-button']")
continue_button.click()


length of # entrys: 15
form element prior to writing answer: ID: input-1, tag: input, entry label: How Did You Hear About Us?, answer: LinkedIn, Other, OTR, answer type: MULTISELECT
made it here
Have you ever worked for JCI or one of its affiliates?
form element prior to writing answer: ID: input-2, tag: div, entry label: Have you ever worked for JCI or one of its affiliates?, answer: No, answer type: None
form element prior to writing answer: ID: input-3, tag: button, entry label: Country / Territory, answer: United States of America, USA, US, answer type: None
form element prior to writing answer: ID: input-4, tag: input, entry label: First Name, answer: John, answer type: TEXTINPUT
form element prior to writing answer: ID: input-5, tag: input, entry label: Last Name, answer: Smith, answer type: TEXTINPUT
form element prior to writing answer: ID: input-6, tag: input, entry label: , answer: , answer type: CHECKBOX
form element prior to writing answer: ID: input-7, tag: input, entry la

In [69]:
#get question/answer database
df_work = pd.read_excel('Answers.xlsx',sheet_name="WorkExp1")
df_work.head(6)

,Questions,Answers
0,Job Title,Investigator
1,Company,Investigations Inc
2,Location,1234 auto st
3,I currently work here,yes
4,From,1/3/2023
5,To,NaN


In [70]:
wb = load_workbook("Answers.xlsx")
sheet_names = wb.get_sheet_names()

work_experience_sheets = [name for name in sheet_names if name.startswith("WorkExp")]
num_work_experience_sheets = len(work_experience_sheets)

print(f"There are {num_work_experience_sheets} sheets that start with the title 'workExperience'.")


There are 2 sheets that start with the title 'workExperience'.


C:\Users\JoshG\AppData\Local\Temp\ipykernel_15696\3438172813.py:2: DeprecationWarning: Call to deprecated function get_sheet_names (Use wb.sheetnames).
  sheet_names = wb.get_sheet_names()


In [71]:
def fill_section(driver, df, exp_container):
    # locate all input elements in work experience 1
    inputs_list = exp_container.find_elements_by_css_selector("[id^='input-']")

    #remove any elements that don't have their id ending in a number
    new_inputs_list = []
    for i in range(len(inputs_list)):
        current_id = inputs_list[i].get_attribute('ID')
        if current_id[-1].isdigit() != False:
            new_inputs_list.append(inputs_list[i])
    inputs_list = new_inputs_list

    print(len(inputs_list))

    #move through each form entry and fill in
    entries = []
    for element in inputs_list:
        #make sure element still exists
        try:
            if len(driver.find_elements_by_xpath(f"//*[@id='{element.get_attribute('ID')}']")) > 0:
                pass
        except:
            continue
        
        #create element object
        entry = FormEntry(element)
        
        #get the type of answer form it is
        entry.get_id()
        entry.get_tag()
        entry.get_answer_type()
        entry.get_label()

        #skip if a span element
        if entry.tag == 'span':
            continue

        #pull answer from form
        answer = get_answer(df,entry.label)
        entry.add_answer(answer)

        print(entry)

        #fill in answer
        entry.update_element(driver)
        entry.write_answer(driver)
            
        #append entry to entries list
        entries.append(entry)

In [79]:
#|export
#
#2nd page of entry forms
load_page_by_dataautomationid('myExperiencePage')

#delete any work experience
delete_exp_btn = driver.find_elements_by_xpath(f"//button[starts-with(@aria-label,'Delete Work Experience')]")
if len(delete_exp_btn) > 0:
    for i in range(len(delete_exp_btn)):
        delete_exp_btn[i].click()

#click work experience button
add_exp_btn = driver.find_elements_by_xpath(f"//button[@aria-label='Add Work Experience']")
add_exp_btn[0].click()

#fill in each form for each job
for i in range(num_work_experience_sheets):
    #get question/answer database
    df_work = pd.read_excel('Answers.xlsx',sheet_name=f"WorkExp{i+1}", dtype=str)

    #get the container for work experience
    exp_container = driver.find_element_by_xpath(f"//div[@data-automation-id='workExperience-{i+1}']")

    #fill in work experience section
    fill_section(driver,df_work,exp_container)

    #click on add another so long as not last work experience to add
    if i != num_work_experience_sheets:  
        #click work experience button
        add_exp_btn = driver.find_elements_by_xpath(f"//button[@data-automation-id='Add Another']")
        add_exp_btn[0].click()




7
ID: input-596, tag: input, entry label: Job Title, answer: Investigator, answer type: TEXTINPUT
ID: input-597, tag: input, entry label: Company, answer: Investigations Inc, answer type: TEXTINPUT
ID: input-598, tag: input, entry label: Location, answer: 1234 auto st, answer type: TEXTINPUT
ID: input-599, tag: input, entry label: I currently work here, answer: yes, answer type: CHECKBOX
ID: input-602, tag: div, entry label: From, answer: 1/3/2023, answer type: DATE_MONTH_YEAR
date to input: 1/3/2023, ID: input-602
12023
ID: input-606, tag: textarea, entry label: Role Description, answer: Work super hard, trust me bro, answer type: TEXTAREA


ElementClickInterceptedException: Message: element click intercepted: Element <button aria-label="... Work Experience" data-automation-id="Add Another" font-size="14" height="40" class="css-2g95d">Add Another</button> is not clickable at point (216, 996). Other element would receive the click: <ul class="css-vcngj">...</ul>
  (Session info: chrome=109.0.5414.120)


In [80]:
wb = load_workbook("Answers.xlsx")
sheet_names = wb.get_sheet_names()

education_sheets = [name for name in sheet_names if name.startswith("Education")]
num_education_sheets = len(work_experience_sheets)

print(f"There are {num_education_sheets} sheets that start with the title 'Education'.")

There are 2 sheets that start with the title 'Education'.


C:\Users\JoshG\AppData\Local\Temp\ipykernel_15696\138365976.py:2: DeprecationWarning: Call to deprecated function get_sheet_names (Use wb.sheetnames).
  sheet_names = wb.get_sheet_names()


In [82]:
#delete any education experience
delete_ed_btn = driver.find_elements_by_xpath(f"//button[starts-with(@aria-label,'Delete Education')]")

if len(delete_ed_btn) > 0:
    print(range(len(delete_exp_btn)+1))
    for i in range(len(delete_exp_btn)+1):
        print("made it here")
        delete_ed_btn = driver.find_elements_by_xpath(f"//button[starts-with(@aria-label,'Delete Education')]")
        delete_ed_btn[0].click()

#click add  education button
add_ed_btn = driver.find_elements_by_xpath(f"//button[@aria-label='Add Education']")
add_ed_btn[0].click()

#fill in each form for each education
for i in range(num_education_sheets):
    #get question/answer database
    df_ed = pd.read_excel('Answers.xlsx',sheet_name=f"Education{i+1}", dtype=str)

    #get the container for education
    ed_container = driver.find_element_by_xpath(f"//div[@data-automation-id='education-{i+1}']")

    #fill in education section
    fill_section(driver,df_ed,ed_container)

    #click on add another so long as not last work experience to add
    if i+1 != num_education_sheets:  
        #click work experience button
        add_ed_btn = driver.find_elements_by_xpath(f"//div[@data-automation-id='educationSection']/descendant::button[@data-automation-id='Add Another']")
        add_ed_btn[0].click()

6
ID: input-695, tag: input, entry label: School or University, answer: California State Polytechnic University Pomona, answer type: TEXTINPUT
ID: input-696, tag: button, entry label: Degree, answer: Bachelors Degree, answer type: None
ID: input-697, tag: input, entry label: Field of Study, answer: Mechanical engineering, answer type: MULTISELECT
made it here
ID: input-698, tag: input, entry label: Overall Result (GPA), answer: 3.5, answer type: TEXTINPUT
ID: input-701, tag: div, entry label: From, answer: 01/01/2019, answer type: DATE_YEAR
2019
ID: input-704, tag: div, entry label: To (Actual or Expected), answer: 02/05/2022, answer type: DATE_YEAR
2022
6
ID: input-771, tag: input, entry label: School or University, answer: Example University, answer type: TEXTINPUT
ID: input-772, tag: button, entry label: Degree, answer: Bachelors Degree, answer type: None
ID: input-773, tag: input, entry label: Field of Study, answer: Accounting, answer type: MULTISELECT
made it here
ID: input-774, 

In [136]:
import wd_classes
from wd_classes import FormEntry
import importlib
importlib.reload(wd_classes)
from wd_classes import FormEntry

In [111]:
def upload_file(file_path):
    # Locate the upload button using its ID or CSS selector
    upload_button = driver.find_element_by_xpath(f"//input[@data-automation-id='file-upload-input-ref']")
    upload_button.send_keys(file_path)

file_path = "C:\\Users\\JoshG\\Documents\\Machine Learning\\JobApp_Automation\\Sample Resume.docx"
upload_file(file_path)

In [164]:
def go_to_next():
    continue_btn = driver.find_element_by_xpath("//button[@data-automation-id='bottom-navigation-next-button']")
    continue_btn.click()

go_to_next()

In [144]:
df = pd.read_excel('Answers.xlsx',sheet_name="Answers", dtype=str)

section = driver.find_element_by_xpath("//div[@data-automation-id='primaryQuestionnairePage']")
fill_section(driver,df,section)


5
ID: input-92, tag: button, entry label: Are you at least 18 years of age? (If not, your employment is subject to verification that you are of at least legal age and that you are able to supply any required work permit)., answer: Yes, answer type: LISTBOX
made it click
ID: input-94, tag: button, entry label: Are you legally eligible to work in the country to which you are applying?, answer: Yes, answer type: LISTBOX
made it click
ID: input-96, tag: button, entry label: Will you now or in the future require sponsorship for employment Visa status to be eligible to work in the country to which you are applying?(e.g., H-1B Visa status/Work Visa/Work Permit)?, answer: No, answer type: LISTBOX
made it click
ID: input-98, tag: button, entry label: Have you ever been granted a government Security Clearance?, answer: No, answer type: LISTBOX
made it click
ID: input-100, tag: button, entry label: Have you signed an employment agreement or any kind of restrictive covenant, covenant not to compet

In [163]:
df = pd.read_excel('Answers.xlsx',sheet_name="RacistQuestions", dtype=str)

section = driver.find_element_by_xpath("//div[@data-automation-id='voluntaryDisclosuresPage']")
fill_section(driver,df,section)

5
ID: input-101, tag: button, entry label: Please select your gender which most accurately describes how you identify yourself., answer: Male, answer type: LISTBOX
made it click
ID: input-102, tag: button, entry label: Please select your race which most accurately describes how you identify yourself., answer: White (United States of America), answer type: LISTBOX
made it click
ID: input-103, tag: button, entry label: Please indicate if you are Hispanic or Latino., answer: No, answer type: LISTBOX
made it click
ID: input-104, tag: button, entry label: Please select the veteran status which most accurately describes your status., answer: I AM NOT A VETERAN, answer type: LISTBOX
made it click
ID: input-105, tag: input, entry label: I certify that I have read, fully understand and accept all terms of the foregoing statement., answer: Yes, answer type: CHECKBOX
